# Neale diseases genes (Pascal) and vascular phenotypes genes (Pascal)
## Unfinished!

###### First version 17/06/2022
###### Latest modif 11/10/2022

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import seaborn as sns
from datetime import datetime
import os
import glob

DATE = datetime.now().strftime("%Y-%m-%d")

In [2]:
p_value_min = 5.7  # -math.log10(0.05/len(x))
ventile_num='ventile4'
date_used='2022_08_17'
directory_genes='/NVME/decrypted/scratch/multitrait/UK_BIOBANK_ZERO/gwas/'+str(date_used)+'_'+str(ventile_num) +'/'
save_dir= '/NVME/decrypted/scratch/multitrait/UK_BIOBANK_ZERO/gwas/gcorr_diseases/'

all_files=['tau1_artery', 'tau1_vein', 'ratio_AV_DF', 'D_A_std', 'D_V_std',
'bifurcations', 'VD_orig_artery', 'VD_orig_vein', 'ratio_VD', 'mean_angle_taa',
'mean_angle_tva', 'eq_CRAE', 'eq_CRVE', 'ratio_CRAE_CRVE', 'medianDiameter_artery',
'medianDiameter_vein', 'ratio_AV_medianDiameter']

neale_csv_name='11_10_2022_diseases_genes.csv'

In [3]:
df_genes_neale= pd.read_csv('/SSD/home/sofia/retina-phenotypes/complementary/GWAS_postprocessing/'+neale_csv_name, converters={'names': pd.eval})

#df_genes_neale['liststring'] = [''.join(map(str, l)) for l in df_genes_neale['names']]
#df_genes_neale['names'].iloc[4][2]
#list_pheno_neale=['4079_irnt'] #list(df_genes_neale['pheno'])
#['4080_irnt', '102_irnt', '4056_irnt', '5610_3', '4079_irnt', '1717', '3894_irnt']

#df_genes_neale_red = df_genes_neale[df_genes_neale["name_pheno"].isin(list_pheno_neale)]
#df_genes_neale_red['pheno_name'] = ['SBP_irnt', 'PR_irnt', 'Stroke_irnt', 'Presbyopia_3', 'DBP_irnt', 'Skin_color', 'heart_attack_irnt']


In [4]:
df_genes_neale

,pheno,name_pheno,N genes,names
0,only_rs_notna_mod_variants_5843_1,5843_1,1,[EXOSC4]
1,only_rs_notna_mod_variants_1717,1717,468,"[CDK10, RP11-368I7.4, DPEP1, CHMP1A, SPATA33, ..."
2,only_rs_notna_mod_variants_4080_irnt,4080_irnt,959,"[NPPA, NPPB, MTHFR, CLCN6, C1orf167, NPR3, AGT..."
3,only_rs_notna_mod_variants_5855_1,5855_1,0,[]
4,only_rs_notna_mod_variants_4056_irnt,4056_irnt,0,[]
5,only_rs_notna_mod_variants_4079_irnt,4079_irnt,910,"[CSK, RP1-278O22.1, LMAN1L, CYP1A2, CPLX3, ULK..."
6,only_rs_notna_mod_variants_4079_raw,4079_raw,905,"[NPPA, NPPB, MTHFR, CLCN6, C1orf167, RP1-278O2..."
7,only_rs_notna_mod_variants_3894_irnt,3894_irnt,0,[]
8,only_rs_notna_mod_variants_5843_3,5843_3,0,[]
9,only_rs_notna_mod_variants_3627_irnt,3627_irnt,0,[]


In [5]:
#### TO DO: Plot the results in a better way once it is decided!

In [6]:
df_aux=pd.DataFrame([])
l_genes_shared_all= []
for file in all_files:
    # Read csvs
    df = pd.read_csv(directory_genes+file+'__gene_scores', delimiter='\t', names =['gen', 'p']) #, index_col=None, header=0)
    df['file_col']=file

    # From p to -log10(p)
    df['-log10(p)'] = -np.log10(df['p'])
    y = df[df['-log10(p)'] >= p_value_min]
    df_significant = y.sort_values('-log10(p)', ascending=False)
    #phenotype_genes=df_significant['gen'].to_list()
    set_pheno=set(df_significant['gen'])
    print(file)
    for i in range(len(df_genes_neale)):
        if len(df_genes_neale['names'].iloc[i]) !=0: 
            set_neale=set(df_genes_neale['names'].iloc[i])
           # print(file, df_genes_neale['name_pheno'].iloc[i])
            print(df_genes_neale['name_pheno'].iloc[i], len(set_neale),len(set_pheno & set_neale))
            #print(len(set_pheno), len(set_neale),len(set_pheno & set_neale))
            #print('Num of genes in pheno, neale, intersection: ',len(set_pheno), len(set_neale),len(set_pheno & set_neale))
            
            #print('Name of the genes intersection: ', set_pheno & set_neale, '\n')
        else:
            continue
    print('\n')
    

tau1_artery
5843_1 1 0
1717 468 13
4080_irnt 959 32
4079_irnt 910 20
4079_raw 905 19
102_raw 884 15
21021_irnt 3 0
1558 276 0
102_irnt 907 20
4080_raw 838 31
2976_irnt 106 7


tau1_vein
5843_1 1 0
1717 468 0
4080_irnt 959 13
4079_irnt 910 11
4079_raw 905 11
102_raw 884 40
21021_irnt 3 1
1558 276 7
102_irnt 907 40
4080_raw 838 16
2976_irnt 106 2


ratio_AV_DF
5843_1 1 0
1717 468 10
4080_irnt 959 31
4079_irnt 910 21
4079_raw 905 21
102_raw 884 7
21021_irnt 3 0
1558 276 0
102_irnt 907 12
4080_raw 838 29
2976_irnt 106 0


D_A_std
5843_1 1 0
1717 468 8
4080_irnt 959 6
4079_irnt 910 2
4079_raw 905 2
102_raw 884 3
21021_irnt 3 0
1558 276 1
102_irnt 907 4
4080_raw 838 6
2976_irnt 106 0


D_V_std
5843_1 1 0
1717 468 13
4080_irnt 959 85
4079_irnt 910 94
4079_raw 905 94
102_raw 884 14
21021_irnt 3 0
1558 276 12
102_irnt 907 14
4080_raw 838 87
2976_irnt 106 25


bifurcations
5843_1 1 0
1717 468 2
4080_irnt 959 5
4079_irnt 910 2
4079_raw 905 2
102_raw 884 4
21021_irnt 3 0
1558 276 1
102_irnt 907 4


In [36]:
#df_genes_count[df_genes_count['Unnamed: 0']=='CTD-2008P7.9']

,Unnamed: 0,gen,ratio_N_pheno
